In [2]:
import pandas as pd
import json
import requests

city_bikes_data = pd.read_csv("C:/Users/colte/LHL-final-project/data/stations_df.csv")

In [13]:
# Set latitude and longitude for params
latitude = city_bikes_data['latitude'].iloc[0]
longitude = city_bikes_data['longitude'].iloc[0]

# Foursquare

Send a request to Foursquare with a small radius (1000m) for all the bike stations in your city of choice. 

In [49]:
# Foursquare API Key
FOURSQUARE_KEY = 'fsq3nRLHnLQh2Y02PDBx8qcYaOIKIzeNRUXditJZC30vLIU='

# Foursquare url
foursquare_url = "https://api.foursquare.com/v3/places/search"

# Foursquare categories for params
foursquare_categories = "13065,13003,17043,17048"

# Foursquare header
foursquare_headers = {
    "accept": "application/json",
    "Authorization": f"{FOURSQUARE_KEY}"
}

# Create Foursquare empty list
foursquare_results = []

# Iterate through first 5 rows
for _, row in city_bikes_data.iterrows():
    latitude = row['latitude']
    longitude = row['longitude']

    # Foursquare params
    foursquare_params = {
    "ll": f"{latitude},{longitude}",
    "radius": 1000,
    "limit": 10,
    "categories": foursquare_categories,
    "sort": "DISTANCE"
    }
    
    # Call API
    foursquare_response = requests.get(foursquare_url, headers=foursquare_headers, params=foursquare_params)

    if foursquare_response.status_code == 200:
        foursquare_data = foursquare_response.json().get("results", [])
        for business in foursquare_data:
            foursquare_results.append({
                "station_latitude": latitude,
                "station_longitude": longitude,
                "poi_name": business.get("name"),
                "poi_address": business.get("location", {}).get("address", "N/A"),
                "poi_categories": ", ".join([cat["short_name"] for cat in business.get("categories", [])]),
                "poi_latitude": business.get("geocodes", {}).get("main", {}).get("latitude", "N/A"),
                "poi_longitude": business.get("geocodes", {}).get("main", {}).get("longitude", "N/A"),
                "distance_from_station": business.get("distance", "N/A")
            })
    else:
        print(f"Failed to fetch data for latitude {latitude}, longitude {longitude}: {foursquare_response.status_code}")


# print(foursquare_response.text) # Used for checking code worked correctly when it was run for just first row

Parse through the response to get the POI (such as restaurants, bars, etc) details you want (ratings, name, location, etc)

In [105]:
# # Used for testing code on first row
# # Put response data into a dict
# foursquare_data = foursquare_response.json()

# # Used for testing code on first row
# # Find the appropriate fields for business name, category, address, and rating
# for business in foursquare_data.get("results", []):
#     name = business.get("name")
#     categories = [category["short_name"] for category in business.get("categories", [])]
#     rating = business.get("rating", 'N/A')
#     address = business.get("location", {}).get("address", "N/A")
#     print(f"name: {name}, address: {address}, rating: {rating}, categories: {', '.join(categories)}")



name: Kortteliravintola Hugo, address: Stålarminkatu 2, rating: N/A, categories: Bar, Gastropub
name: Vaate & Paikka, address: N/A, rating: N/A, categories: Apparel
name: Kasperin Yksiö, address: N/A, rating: N/A, categories: Hotel Bar


Put your parsed results into a DataFrame

In [53]:
# # Used for testing code on first row
# # Extract business list
# foursquare_businesses = foursquare_data.get("results", [])

# # Used for testing code on first row
# #Create list of required fields from the foursquare dict
# foursquare_list = [
#     {
#         "name": business.get("name"),
#         "address": business.get("location", {}).get("address", "N/A"),
#         "categories": ", ".join([category["short_name"] for category in business.get("categories", [])]),
#         "rating": business.get("rating", None),
#         "latitude": business.get("geocodes", {}).get("main", {}).get("latitude", "N/A"),
#         "longitude": business.get("geocodes", {}).get("main", {}).get("longitude", "N/A")
#     }
#     for business in foursquare_businesses
# ]

# # Used for testing code on first row
# # Convert list to df
# foursquare_df = pd.DataFrame(foursquare_list)

# Convert results to a DataFrame
foursquare_df = pd.DataFrame(foursquare_results)

# Display the results
print(foursquare_df.head(2))

# Create file path for foursquare df
foursquare_file_path = "C:/Users/colte/LHL-final-project/data/foursquare_df.csv"

# Add foursquare_df.csv to file path
foursquare_df.to_csv(foursquare_file_path, index=False)

print(f"df was saved to {foursquare_file_path}")

   station_latitude  station_longitude poi_name   poi_address poi_categories  \
0         60.441012           22.25659    Mylly  Martinkatu 7            Bar   
1         60.441012           22.25659     Uuni           N/A          Pizza   

   poi_latitude  poi_longitude  distance_from_station  
0     60.441665      22.257414                     85  
1     60.443080      22.256905                    230  
df was saved to C:/Users/colte/LHL-final-project/data/foursquare_df.csv


# Yelp

Send a request to Yelp with a small radius (1000m) for all the bike stations in your city of choice. 

In [55]:
# Yelp API Key
YELP_KEY = 'ny8E6_YvMJMaxR8895q-m4dxWtNwB0HaxfYWA78EJ0xKTlhPHTo6hbN-oqeDea6SYJcR6xHtBGnmXm5QH8n3ohjlg63LkCG5BGgVkJppyyLJ1N08bdYbCzsTGPtYZ3Yx'

# Yelp url
yelp_url = "https://api.yelp.com/v3/businesses/search"

# Yelp categories for params
yelp_categories = "restaurants, bars, streetwear, sneakers, clothing, shoe"

#Yelp header
yelp_headers = {
    "accept": "application/json",
    "Authorization": f"Bearer {YELP_KEY}"
}


# # Used for testing code on first row
# yelp_params = {
#     "sort_by": "distance",
#     "limit": 10,
#     "radius": 1000,
#     "latitude": latitude,
#     "longitude": longitude,
#     "categories": yelp_categories
# }

# # Used for testing code on first row
# response = requests.get(yelp_url, headers=yelp_headers, params=yelp_params)

# # Used for testing code on first row
# print(response.text)

# Create Yelp empty list
yelp_results = []

# Iterate through first 5 rows
for _, row in city_bikes_data.iterrows():
    latitude = row['latitude']
    longitude = row['longitude']

    # Yelp params
    yelp_params = {
        "sort_by": "distance",
        "limit": 10,
        "radius": 1000,
        "latitude": latitude,
        "longitude": longitude,
        "categories": yelp_categories
    }
    
    # Call API
    yelp_response = requests.get(yelp_url, headers=yelp_headers, params=yelp_params)

    if yelp_response.status_code == 200:
        yelp_data = yelp_response.json().get("businesses", [])
        for business in yelp_data:
            yelp_results.append({
                "station_latitude": latitude,
                "station_longitude": longitude,
                "poi_name": business.get("name"),
                "poi_address": business.get("location", {}).get("address1", "N/A"),
                "poi_categories": ", ".join([category["title"] for category in business.get("categories", [])]),
                "poi_rating": business.get("rating", None),
                "poi_latitude": business.get("coordinates", {}).get("latitude", "N/A"),
                "poi_longitude": business.get("coordinates", {}).get("longitude", "N/A")
            })
    else:
        print(f"Failed to fetch data for latitude {latitude}, longitude {longitude}: {yelp_response.status_code}")


Parse through the response to get the POI (such as restaurants, bars, etc) details you want (ratings, name, location, etc)

In [78]:
# # Used for testing code on first row
# # Put response data into a dict
# yelp_data = response.json()

# # Used for testing code on first row
# # Find the appropriate fields for business name, category, address, and rating
# for business in yelp_data.get("businesses", []):
#     name = business.get("name")
#     categories = [category["title"] for category in business.get("categories", [])]
#     rating = business.get("rating", 'N/A')
#     address = business.get("location", {}).get("address1", "")
#     print(f"name: {name}, address: {address}, rating: {rating}, categories: {', '.join(categories)}")

name: Pub Hopotus, address: Kuninkaankartanonkatu 8, rating: 0.0, categories: Restaurants
name: Martinmäen kotiruokapalvelu Avoin, address: Tiilentekijänkatu 10J, rating: 0.0, categories: Restaurants
name: Naantalin Aurinkoinen, address: Martinkatu 3, rating: 0.0, categories: Restaurants
name: Melange, address: Martinkatu 3, rating: 0.0, categories: Restaurants
name: Maku Turku, address: Stålarminkatu 5, rating: 2.5, categories: Pan Asian
name: Ravintola Pippurimylly, address: Stålarminkatu 2, rating: 4.7, categories: Steakhouses
name: designed by oodi pr, address: Tiilentekijänkatu 3B A, rating: 0.0, categories: Restaurants
name: Haarala Tiina, address: Itäinen Rantakatu 48H 242, rating: 0.0, categories: Restaurants
name: Ravintola-Pizzeria Ranta-Osku avoin, address: Martinkatu 1, rating: 0.0, categories: Restaurants
name: T:mi Maryam-M, address: Martinkatu 1, rating: 0.0, categories: Restaurants


Put your parsed results into a DataFrame

In [59]:
# # Used for testing code on first row
# # Extract business list
# yelp_businesses = yelp_data.get("businesses", [])

# # Used for testing code on first row
# #Create list of required fields from the yelp dict
# yelp_list = [
#     {
#         "name": business.get("name"),
#         "address": business.get("location", {}).get("address1", ""),
#         "categories": ", ".join([category["title"] for category in business.get("categories", [])]),
#         "rating": business.get("rating", None),
#         "latitude": business.get("coordinates", {}).get("latitude"),
#         "longitude": business.get("coordinates", {}).get("longitude")
#     }
#     for business in yelp_businesses
# ]

# # Used for testing code on first row
# # Convert list to df
# yelp_df = pd.DataFrame(yelp_list)

# Convert results to a DataFrame
yelp_df = pd.DataFrame(yelp_results)

# Display the results
print(yelp_df.head(2))

# Create file path for yelp df
yelp_file_path = "C:/Users/colte/LHL-final-project/data/yelp_df.csv"

# Add yelp_df.csv to file path
yelp_df.to_csv(yelp_file_path, index=False)

print(f"df was saved to {yelp_file_path}")

   station_latitude  station_longitude                           poi_name  \
0         60.441012           22.25659                        Pub Hopotus   
1         60.441012           22.25659  Martinmäen kotiruokapalvelu Avoin   

               poi_address poi_categories  poi_rating  poi_latitude  \
0  Kuninkaankartanonkatu 8    Restaurants         0.0     60.439449   
1    Tiilentekijänkatu 10J    Restaurants         0.0     60.439629   

   poi_longitude  
0      22.259060  
1      22.253693  
df was saved to C:/Users/colte/LHL-final-project/data/yelp_df.csv


# Comparing Results

Which API provided you with more complete data? Provide an explanation. 

Get the top 10 restaurants according to their rating

In [79]:
top_rated = yelp_df[["poi_name", "poi_address", "poi_rating"]].sort_values(by="poi_rating", ascending=False).head(10)
top_rated = top_rated.reset_index(drop=True)

print(top_rated)

                     poi_name             poi_address  poi_rating
0            Orikedon Taverna  Vanha Tampereentie 137         5.0
1  Yangtze Chinese Restaurant            Hämeenkatu 9         5.0
2                      Sahara       Aninkaistenkatu 5         5.0
3                Roster Turku      Tuomiokirkonkatu 6         5.0
4  Yangtze Chinese Restaurant            Hämeenkatu 9         5.0
5           Ravintola Carlina       Aninkaistenkatu 3         5.0
6                 Bagel House      Yliopistonkatu 3 0         5.0
7      Dalia Kebab & Pizzeria            Martinkatu 1         5.0
8       Pink Vanilla Desserts          Kauppiaskatu 9         5.0
9            Orikedon Taverna  Vanha Tampereentie 137         5.0
